In [2]:
from langsmith import Client
import os
from langchain.smith import RunEvalConfig, run_on_dataset
from langchain_openai import ChatOpenAI
import pandas as pd

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
print("当前目录是：", os.getcwd())
# 验证是否正确加载
print("Langsmith API Key:", os.getenv("LANGSMITH_API_KEY"))
print("Langsmith Project:", os.getenv("LANGSMITH_PROJECT"))

当前目录是： /Users/chen/Desktop/Code/tutorials
Langsmith API Key: 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
Langsmith Project: test-project-langgraph


In [4]:

llm = ChatOpenAI()
response = llm.invoke("Hello, world!")
print(response.content)

Hello! How can I assist you today?


In [5]:
# 创建 LangSmith 客户端
client = Client()

# 尝试列出数据集，验证 API 是否正常工作
try:
    datasets = client.list_datasets()
    print("✅ 成功连接到 LangSmith API！可用数据集数量：", len(list(datasets)))
except Exception as e:
    print("❌ 无法连接到 LangSmith API")
    print("错误信息:", e)

✅ 成功连接到 LangSmith API！可用数据集数量： 1


In [ ]:
# 1. Create a Dataset (Only Inputs, No Output)

example_inputs = [
    "a rap battle between Atticus Finch and Cicero",
    "a rap battle between Barbie and Oppenheimer",
    "a Pythonic rap battle between two swallows: one European and one African",
    "a rap battle between Aubrey Plaza and Stephen Colbert",
]

dataset_name = "Rap Battle Dataset"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Rap battle prompts.",
)

for question in example_inputs:
    # Each example must be unique and have inputs defined.
    # Outputs are optional
    client.create_example(
        inputs={"question": question},
        outputs=None,
        dataset_id=dataset.id,
    )

In [ ]:
# 2. Evaluate Datasets with LLM

eval_config = RunEvalConfig(
    evaluators=[
        # You can specify an evaluator by name/enum.
        # In this case, the default criterion is "helpfulness"
        "criteria",
        # Or you can configure the evaluator
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria("misogyny"),
        RunEvalConfig.Criteria(
            {
                "cliche": "Are the lyrics cliche? "
                "Respond Y if they are, N if they're entirely unique."
            }
        ),
    ]
)

run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    evaluation=eval_config,
)

# Different Ways of Creating Datasets in LangSmith

In [6]:
# 1. Create a Dataset From a List of Examples (Key-Value Pairs)

example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptiles known for?", "Having scales"),
    (
        "What's the main characteristic of amphibians?",
        "They live both in water and on land",
    ),
]

dataset_name = "Elementary Animal Questions"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions and answers about animal phylogenetics.",
)

for question, answer in example_inputs:
    client.create_example(
        inputs={"question": question},
        outputs={"answer": answer},
        dataset_id=dataset.id,
    )

In [ ]:
# 3. Create a Dataset From a Dataframe

# Create a Dataframe

example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptiles known for?", "Having scales"),
    (
        "What's the main characteristic of amphibians?",
        "They live both in water and on land",
    ),
]

df_dataset = pd.DataFrame(example_inputs, columns=["Question", "Answer"])
df_dataset.head()

In [ ]:
input_keys = ["Question"]
output_keys = ["Answer"]

# Create Dataset

dataset = client.upload_dataframe(
    df=df_dataset,
    input_keys=input_keys,
    output_keys=output_keys,
    name="My Dataframe Dataset",
    description="Dataset created from a dataframe",
    data_type="kv",  # The default
)

In [ ]:
# 4. Create a Dataset From a CSV File

# Save the Dataframe as a CSV File

csv_path = "./data/dataset.csv"
df_dataset.to_csv(csv_path, index=False)

# Create Dataset

dataset = client.upload_csv(
    csv_file=csv_path,
    input_keys=input_keys,
    output_keys=output_keys,
    name="My CSV Dataset",
    description="Dataset created from a CSV file",
    data_type="kv",
)

# Correctness: LangSmith Question-Answer Evaluation

In [ ]:
# 1. Evaluate Datasets That Contain Labels

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",  # correctness: right or wrong
        "context_qa",  # refer to example outputs
        "cot_qa",  # context_qa + reasoning
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Elementary Animal Questions",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

In [7]:
# 2. Evaluate Datasets With Customized Criterias

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.LabeledCriteria(
            {
                "helpfulness": (
                    "Is this submission helpful to the user,"
                    " taking into account the correct reference answer?"
                )
            }
        ),
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Elementary Animal Questions",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'advanced-mom-49' at:
https://smith.langchain.com/o/71f16ca1-1af3-4c25-8798-55db383a73f5/datasets/1f318077-f55f-4498-87f9-f7770a31bc01/compare?selectedSessions=acd2e84b-bfcd-46db-b37b-243b431908ad

View all tests for Dataset Elementary Animal Questions at:
https://smith.langchain.com/o/71f16ca1-1af3-4c25-8798-55db383a73f5/datasets/1f318077-f55f-4498-87f9-f7770a31bc01
[------------------------------------------------->] 4/4

{'project_name': 'advanced-mom-49',
 'results': {'038ad58c-7f11-43ed-b8b0-6d82033e5e90': {'input': {'question': "What's the main characteristic of amphibians?"},
   'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment="The criterion for this task is the helpfulness of the submission, taking into account the correct reference answer. \n\nThe reference answer states that the main characteristic of amphibians is that they live both in water and on land. \n\nThe AI's submission includes this information, stating that amphibians have a dual life cycle and typically live both on land and in water. \n\nIn addition to this, the AI's submission provides further information about amphibians, such as the fact that they are cold-blooded, have moist, smooth skin, begin life as water-breathing larvae, metamorphose into adults capable of breathing air, and lay eggs in water. \n\nThis additional information is relevant and accurate, and it enhances the helpfulness of the submis

In [ ]:
# 3. Evaluate Datasets Without Labels

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.Criteria(
            {"creativity": "Is this submission creative, imaginative, or novel?"}
        ),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Rap Battle Dataset",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

In [ ]:
# 4. Evaluate Datasets Based on Cosine Distance Criteria
# Cosine Distance: Ranged Between 0 to 1. 0 = More Similar

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        "embedding_distance",
        # Or to customize the embeddings:
        # Requires 'pip install sentence_transformers'
        # RunEvalConfig.EmbeddingDistance(embeddings=HuggingFaceEmbeddings(), distance_metric="cosine"),
    ]
)

run_on_dataset(
    client=client,
    dataset_name="Elementary Animal Questions",
    llm_or_chain_factory=llm,
    evaluation=evaluation_config,
)

In [31]:
# 删除数据集

# 获取所有数据集
datasets = list(client.list_datasets())
print(f"当前数据集: {datasets} ")

# ✅ 批量删除
for ds in datasets:
    print(f"🗑️ 删除数据集: {ds.name} (ID: {ds.id})")
    client.delete_dataset(dataset_id = ds.id)


当前数据集: [Dataset(name='Elementary Animal Questions', description='Questions and answers about animal phylogenetics.', data_type=<DataType.kv: 'kv'>, id=UUID('d8682549-f339-411b-9772-261bdddff364'), created_at=datetime.datetime(2025, 6, 26, 5, 36, 38, 26943, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 6, 26, 5, 36, 38, 26943, tzinfo=datetime.timezone.utc), example_count=4, session_count=3, last_session_start_time=datetime.datetime(2025, 6, 26, 5, 44, 5, 953506), inputs_schema=None, outputs_schema=None, transformations=None), Dataset(name='Rap Battle Dataset', description='Rap battle prompts.', data_type=<DataType.kv: 'kv'>, id=UUID('2013dbcd-1ca9-4e41-b1a3-9990b2b4698c'), created_at=datetime.datetime(2025, 6, 26, 5, 13, 36, 904799, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 6, 26, 5, 13, 36, 904799, tzinfo=datetime.timezone.utc), example_count=4, session_count=5, last_session_start_time=datetime.datetime(2025, 6, 26, 5, 43, 43, 101001), inpu